<a href="https://colab.research.google.com/github/Choy98/Mestizo-Restaurant-web/blob/main/notebook4148565786.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'skin-cancer-mnist-ham10000:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F54339%2F104884%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240615%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240615T032528Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D61ec4b3a6973a1945d02bcb736767e76dd8205b09355bbaf8997fcfcfcc5d23571a85db48c80b9a5f1f2c089416a7e6b5df53a992f7bd605a5fc34e93adf4b4558e47c133b5be7985c21f79b44bf7a70f03e0079abd8592b6d3ba1b3c2fc539ff221877d7feba38b0758087d14c6bbcf08e82cd77fa334b9de573ee2cd30cdcf38a783524f238aae7e29bc8277716c64ae3fe39dfa24752e9db9632ed40088ca99db9c03ab417ed96178ed8c792458283fed2c098d84e24d8f4d7eda6b778058bb30722d902204359f9078d24bae2f325dbc35da617d3ef80d747694fc47e8a78f9e20e1ae7278810fd8efbbd725f8acf34f3f6d667019fab3c3614af6876f3b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


### **Loading Used Libriries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import random
import os
import torch ## PyTorch
import torch.nn as nn ## Neural networks package
from torch import optim ## optimizer
from torchvision import models ## package consists of popular datasets, model architectures, and common image transformations for computer vision.
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchvision.transforms as transforms ## for data augmentation
from torchvision.transforms import v2 ## for data augmentation
from sklearn.model_selection import train_test_split ## to split datasets



# Setting the random seed for reproducibility
seed = 77
np.random.seed(seed) ## for numpy
torch.manual_seed(seed) ## for PyTorch
torch.cuda.manual_seed(seed)
random.seed(24) ## for random module on python


# Check if a GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("GPU is not available.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu") ## use gpu if its available

### **1 - EDA of HAM10000 Dataset**

##### **1.1 - Loading Data**

In [ ]:
## Path to data
HAM1000_path = '/kaggle/input/skin-cancer-mnist-ham10000/'

## to map the class acronym to its real name, according to the paper of the dataset
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

## collecting image paths
all_image_path = glob(os.path.join(HAM1000_path, '*', '*.jpg'))

## imageid_path_dict will map each image ID (the file name without the extension) to its full file path.
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}


df_original = pd.read_csv(os.path.join(HAM1000_path, 'HAM10000_metadata.csv'))
df_original['path'] = df_original['image_id'].map(imageid_path_dict.get)
df_original['cell_type'] = df_original['dx'].map(lesion_type_dict.get)
df_original['cell_type_idx'] = pd.Categorical(df_original['cell_type']).codes

## class names
classes = df_original['dx'].unique()

df_original.head()

##### **1.2 - Data Distribution**

In [ ]:
# Count the number of images in each class
class_counts = df_original['cell_type'].value_counts()

# Plot a histogram
plt.figure(figsize=(10, 6))
class_counts.plot(kind='barh')

# Add title and labels
plt.title('Number of Images per Class')
plt.ylabel('Class')
plt.xlabel('Number of Images')

# Display the plot
plt.tight_layout()
plt.show()


print(class_counts)

#### We have a clearly prevalence of the Melanocytic nevi class but, more generally, it is possible to notice an imbalance of the classes samples.

##### **1.3 Samples Example**

In [ ]:
%%time
## Showing some examples of the data

# Randomly select 15 rows from the dataframe
sample_df = df_original.sample(n=15)

# Set up the matplotlib figure and axes
fig, axes = plt.subplots(3, 5, figsize=(20, 12))
axes = axes.flatten()

for ax, (_, row) in zip(axes, sample_df.iterrows()):
    img_path = row['path']
    cell_type = row['cell_type']

    # Open the image
    img = Image.open(img_path)

    # Display the image on the axis
    ax.imshow(img)
    ax.set_title(cell_type)
    ax.axis('off')  # Hide the axes ticks

# Add a main title for the whole plot
plt.suptitle('HAM10000 Examples', fontsize=20)
# Adjust the layout
plt.tight_layout()
plt.show()


### **2 - Data Pre-processing**

##### **2.1 - Useful Functions**

In [ ]:
## Plots the ROC curve
def plot_roc_curve(fpr, tpr, roc_auc):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()


## gets the mean and std from a given dataset
def calculate_mean_std_from_df(df, path_column):
    transform = transforms.ToTensor()

    mean = np.zeros(3)
    std = np.zeros(3)
    nb_samples = 0

    for img_path in df[path_column]:
        img = Image.open(img_path).convert('RGB')
        img = transform(img)
        img = img.view(3, -1)
        mean += img.mean(dim=1).numpy()
        std += img.std(dim=1).numpy()
        nb_samples += 1

    mean /= nb_samples
    std /= nb_samples

    return mean, std

## Define the custom dataset class
class CustomHAM10000(Dataset):
    def __init__(self, dataframe, img_size, transform=None):
        self.paths = dataframe['path'].values
        self.labels = torch.tensor(dataframe['cell_type_idx'].values, dtype=torch.long)
        self.transform = transform
        self.img_size = img_size

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        image = Image.open(img_path)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


##### **2.2 - Split Data**

In [ ]:
## Setting Hyperparameters

img_size = (120,120)  ## Used by https://arxiv.org/pdf/2303.07520
batch_size = 32  ## Default
random_state = seed

In [ ]:
## separates between training (full and reduced), validation (the reference paper doesnt use test set)
df_train, df_val = train_test_split(df_original, test_size=0.11, stratify=df_original['dx'], random_state=random_state)
#df_train, df_val = train_test_split(df_train_val, test_size=0.2, stratify=df_train_val['dx'], random_state=random_state)
df_train_small, _ = train_test_split(df_train, train_size=0.25, stratify=df_train['dx'], random_state=random_state)


## see datasets infos
print('-'*30)
print('datasets division')
print('-'*30)
print(f'Nº train (small) = {len(df_train_small)}')
print(f'Nº train (full)  = {len(df_train)}')
print(f'Nº val           = {len(df_val)}')
#print(f'Nº test          = {len(df_test)}')

#### The test_size was chosen to match the number of samples present on the validation set present on: https://ieeexplore.ieee.org/document/10020302.

In [ ]:
print(df_train['cell_type'].value_counts())
print('-'*40)
print(df_train_small['cell_type'].value_counts())
print('-'*40)
print(df_val['cell_type'].value_counts())
#print('-'*40)
#print(df_test['cell_type'].value_counts())
#print('-'*40)

#### In order to evaluate the model realistically, the validation set will also present an imbalance. We define a small dataset to test different hyperparameters and techniques with time efficiency.

##### **2.3 - Data Augmentation**

In [ ]:
%%time
## Applying data augmentation on classes with lower prevalence

lower_prevalence_df = df_train[df_train['cell_type'] != 'Melanocytic nevi']
high_prevalence_df = df_train[df_train['cell_type'] == 'Melanocytic nevi']

#lower_prevalence_df_small = df_train_small[df_train_small['cell_type'] != 'Melanocytic nevi']
#high_prevalence_df_small = df_train_small[df_train_small['cell_type'] == 'Melanocytic nevi']


## getting the mean and std from the training data
mean_train, std_train = calculate_mean_std_from_df(df_train, 'path')
#mean_train_small, std_train_small = calculate_mean_std_from_df(df_train_small, 'path')

In [ ]:
## Configuring transformer that will perform the Data Augmentation

## transformations to the augmented classes
transform_low_prev_class = transforms.Compose([
                        transforms.RandomRotation(15), transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip(),
                        transforms.RandomAffine(degrees=0, translate=(0.2,0.2)), ##random width height shift
                        transforms.RandomResizedCrop(img_size, scale=(0.7, 1.0)), ##random zoom
                        transforms.ColorJitter(brightness=0.2, contrast=0.2), ## bright and contrast adjustment
                        transforms.Resize(img_size),
                        transforms.ToTensor(), transforms.Normalize(mean_train, std_train)])

## transformations for the rest of the data
normal_transform = transforms.Compose([transforms.Resize(img_size),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean_train, std_train)])

##### **2.4 - Creating the Datasets**

In [ ]:
## Creates the datasets

## train_small_dataset
lowprev_classes_augmented = CustomHAM10000(lower_prevalence_df, img_size, transform=transform_low_prev_class) ## applies data aug only on minority classes
highprev_class_data = CustomHAM10000(high_prevalence_df, img_size, transform=normal_transform)

#train_data_small= ConcatDataset([lowprev_classes_augmented, highprev_class_data])

#ds_train_small = DataLoader(train_data_small, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

## train_dataset

train_data= ConcatDataset([lowprev_classes_augmented, highprev_class_data])

ds_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)


## val_dataset

val_data = CustomHAM10000(df_val, img_size, transform=normal_transform)

ds_val = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True) ## Shuffle = False (Reproducibility)


## test_dataset
#test_data = CustomHAM10000(df_test, img_size, transform=normal_transform)
#ds_test = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)


#### Now, we applied data augmentation to the classes with lower prevalence and created the datasets that the models will be trained and tested on. It is important to notice that the data is normalized, which is a good practice since it enhances performance. The Data Augmentation technique should reduce (but not eliminate) the imbalance problem that we detected earlier in our analysis.

### **3 - Models' Experiments**

##### **3.1 - Importing Pretrained ImageNet Models**

In [ ]:
## Function to load the models

def initialize_net_arch(model_name, num_classes, input_size, feature_extract, use_pretrained = True):

    ## stores the feature extractor of the selected model
    model_backbone = None


    if model_name == 'vgg16':

        model_backbone = models.vgg16(pretrained=True)
        #set_parameter_requires_grad(model_ft, feature_extract) ## for fine tunning

        ## new FC layer
        num_ftrs = model_backbone.classifier[6].in_features
        model_backbone.classifier[6] = nn.Sequential(
            nn.Linear(num_ftrs, 4096), ## dense layer
            nn.ReLU(), ## relu activation
            nn.Dropout(p=0.5), ## dropout with p probability
            nn.Linear(4096, num_classes)
            #nn.Softmax(dim=1) ## softmax activation (no need because of the loss used CrossEntropy)
        )


    elif model_name == 'resnet50':

        model_backbone = models.resnet50(pretrained=use_pretrained)
        #set_parameter_requires_grad(model_backbone, feature_extract) ## for fine tuning

        ## new FC layer
        num_ftrs = model_backbone.fc.in_features
        model_backbone.fc = nn.Sequential(
            nn.Linear(num_ftrs, 4096), ## dense layer
            nn.ReLU(), ## relu activation
            nn.Dropout(p=0.5), ## dropout with p probability
            nn.Linear(4096, num_classes)
        )



    elif model_name == 'inceptionV3':
        '''Beware, the InceptionV3 uses Nx3x299x299 inputs, so you must resize your images to match that'''
        model_backbone = models.inception_v3(pretrained=use_pretrained)
        #set_parameter_requires_grad(model_backbone, feature_extract) ## for fine tuning

        ## new FC layer

        ## handle the auxilary net
        num_ftrs = model_backbone.AuxLogits.fc.in_features
        model_backbone.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)

        ## handle the primary net
        num_ftrs = model_backbone.fc.in_features
        model_backbone.fc = nn.Sequential(
            nn.Linear(num_ftrs, 4096), ## dense layer
            nn.ReLU(), ## relu activation
            nn.Dropout(p=0.5), ## dropout with p probability
            nn.Linear(4096, num_classes)
        )

    elif model_name == 'densenet':
        ## choosing the 161 model because it's closer to the inception performance on the ImageNet dataset

        model_backbone = models.densenet161(pretrained=use_pretrained)

        ## new FC layer
        num_ftrs = model_backbone.classifier.in_features
        model_backbone.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 4096), ## dense layer
            nn.ReLU(), ## relu activation
            nn.Dropout(p=0.5), ## dropout with p probability
            nn.Linear(4096, num_classes)
        )



    else:
        print("Model name not listed, exiting...\n\n")
        exit()


    return model_backbone



In [ ]:
## Setting some parameters

num_classes = 7
feature_extract = True

## Used by https://arxiv.org/pdf/2303.07520
criterion = nn.CrossEntropyLoss()
epochs = 30

##### **3.2 - Defining the Metrics Used**

In [ ]:
## Functions of the metrics
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc


## Accuracy function
def calculate_accuracy(predictions, labels):
    correct = (predictions == labels).sum().item()
    total = labels.size(0)
    return correct / total

## Normalized Multi-class Accuracy function  (Ref -> https://challenge.isic-archive.com/landing/2019/)
def normalized_multiclass_accuracy(predictions, labels, num_classes):
    recall_per_class = []

    for i in range(num_classes):
        true_positives = ((predictions == i) & (labels == i)).sum().item()
        total_positives = (labels == i).sum().item()

        if total_positives > 0:
            recall_per_class.append(true_positives / total_positives)
        else:
            recall_per_class.append(0.0)

    normalized_accuracy = sum(recall_per_class) / num_classes
    return normalized_accuracy

## Funcion that calculates the remaining metrics
def calculate_other_metrics(predictions, labels):
    precision = precision_score(labels.cpu(), predictions.cpu(), average='weighted')
    recall = recall_score(labels.cpu(), predictions.cpu(), average='weighted')
    f1 = f1_score(labels.cpu(), predictions.cpu(), average='weighted')

    # Compute ROC curve and AUC for each class
    num_classes = len(torch.unique(labels))
    fpr = []
    tpr = []
    roc_auc = []
    for class_idx in range(num_classes):
        class_labels = (labels.cpu() == class_idx)
        class_predictions = (predictions.cpu() == class_idx)
        fpr_i, tpr_i, _ = roc_curve(class_labels, class_predictions, pos_label=1)
        roc_auc_i = auc(fpr_i, tpr_i)
        fpr.append(fpr_i)
        tpr.append(tpr_i)
        roc_auc.append(roc_auc_i)

    # Calculate macro-average AUC
    macro_auc = sum(roc_auc) / num_classes

    return precision, recall, f1, macro_auc

##### **3.3 - Defining the Training/Evalutation Loop**

In [ ]:
## Training epoch function
def train_epoch(model, optimizer, data_loader, loss_criterion, num_classes):
    '''Train a Neural Network for one epoch'''
    dev = next(model.parameters()).device ## gets where the model is alocated
    model.train() ## tells that we're training so Dropout, etc will be active.

    running_loss = 0
    all_predictions = []
    all_labels = []

    for images, labels in data_loader:
        images, labels = images.to(dev), labels.to(dev)
        optimizer.zero_grad() ## cleans the gradient tensor (it accumulates)
        output = model(images) ## get prediction of the model
        loss = loss_criterion(output, labels) ## calculate loss
        loss.backward() ## backprop to obtain the influence of the weights on the loss func
        optimizer.step() ## updates the weights

        running_loss += loss.item() ## accumulates the loss to calculate the average loss.

        _, predicted = torch.max(output, 1)
        all_predictions.append(predicted)
        all_labels.append(labels)

    all_predictions = torch.cat(all_predictions)
    all_labels = torch.cat(all_labels)
    accuracy = calculate_accuracy(all_predictions, all_labels)
    normalized_accuracy = normalized_multiclass_accuracy(all_predictions, all_labels, num_classes)

    return running_loss / len(data_loader), accuracy, normalized_accuracy

## Evaluation function
def evaluate(model, data_loader, loss_criterion, num_classes):
    '''Evaluate the model on the validation set'''
    dev = next(model.parameters()).device
    model.eval() ## tells that we're evalutating the model, so we don't use Dropout layers, etc

    running_loss = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(dev), labels.to(dev)
            output = model(images)
            loss = loss_criterion(output, labels)

            running_loss += loss.item()

            _, predicted = torch.max(output, 1) ## gets the predicted class with the highest probability (MSP).
            all_predictions.append(predicted)
            all_labels.append(labels)

    all_predictions = torch.cat(all_predictions)
    all_labels = torch.cat(all_labels)
    accuracy = calculate_accuracy(all_predictions, all_labels)
    normalized_accuracy = normalized_multiclass_accuracy(all_predictions, all_labels, num_classes)

    return running_loss / len(data_loader), accuracy, normalized_accuracy, all_predictions, all_labels


## Training loop function
def train_loop(model, optimizer, criterion, train_loader, val_loader, epochs, num_classes):

    best_val_acc = 0
    best_val_acc_epoch = 1

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")

        train_loss, train_accuracy, train_normalized_accuracy = train_epoch(model, optimizer, train_loader, criterion, num_classes)
        val_loss, val_accuracy, val_normalized_accuracy, val_predictions, val_labels = evaluate(model, val_loader, criterion, num_classes)

        print(f"Training Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train Normalized Accuracy: {train_normalized_accuracy:.4f}")
        print(f"Validation Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val Normalized Accuracy: {val_normalized_accuracy:.4f}")
        print('--'*50)

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            best_val_acc_epoch = epoch

    print(f'Best Results at epoch {best_val_acc_epoch}. Val Accuracy = {best_val_acc}\n\n\n')

    return val_predictions, val_labels

##### **3.4 - Training and Evaluation of Models**

In [ ]:
## Instantiating the models

#vgg16 = initialize_net_arch('vgg16', num_classes, img_size, feature_extract, use_pretrained = True)

#resnet50 = initialize_net_arch('resnet50', num_classes, img_size, feature_extract, use_pretrained = True)

densenet = initialize_net_arch('densenet', num_classes, img_size, feature_extract, use_pretrained = True)

In [ ]:
## Setting the optimizer to each network

## Used by https://arxiv.org/pdf/2303.07520

#optimizer_vgg16 = optim.Adam(vgg16.parameters(), lr=0.0001)
#optimizer_resnet50 = optim.Adam(resnet50.parameters(), lr=0.0001)
optimizer_densenet = optim.Adam(densenet.parameters(), lr=0.0001) ## lr=0.001 (paper)

In [ ]:
## Training/Evaluating on validation set vgg16

## set model to devide
#vgg16 = vgg16.to(device)

#vgg16_val_predictions, vgg16_val_labels = train_loop(vgg16, optimizer_vgg16, criterion, ds_train_small, ds_val, epochs, num_classes)

In [ ]:
## Training/Evaluating on validation set resnet50

## set model to devide
#resnet50 = resnet50.to(device)

#resnet50_val_predictions, resnet50_val_labels = train_loop(resnet50, optimizer_resnet50, criterion, ds_train_small, ds_val, epochs, num_classes)

In [ ]:
%%time
## Training/Evaluating on validation set densenet

## set model to devide
densenet = densenet.to(device)

densenet_val_predictions, densenet_val_labels = train_loop(densenet, optimizer_densenet, criterion, ds_train, ds_val, epochs, num_classes)

In [ ]:
## Seeing the other metrics

#print('VGG-16\n')

#precision, recall, f1, roc_auc = calculate_other_metrics(vgg16_val_predictions, vgg16_val_labels)
#print(f"Weighted Precision: {precision:.4f}, Weighted Recall: {recall:.4f}, Weighted F1: {f1:.4f}, Avg-AUC: {roc_auc:.4f}\n\n")


#print('ResNet50\n')
#precision, recall, f1, roc_auc = calculate_other_metrics(resnet50_val_predictions, resnet50_val_labels)
#print(f"Weighted Precision: {precision:.4f}, Weighted Recall: {recall:.4f}, Weighted F1: {f1:.4f}, Avg-AUC: {roc_auc:.4f}")

print('Densenet\n')
precision, recall, f1, roc_auc = calculate_other_metrics(densenet_val_predictions, densenet_val_labels)
print(f"Weighted Precision: {precision:.4f}, Weighted Recall: {recall:.4f}, Weighted F1: {f1:.4f}, Avg-AUC: {roc_auc:.4f}")